# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Colab Notebooks/Sequential_NLP/Fake News Challenge/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall(path='/content/drive/My Drive/Colab Notebooks/Sequential_NLP/')

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd
import os
#os.chdir('/content/drive/My Drive/Colab Notebooks/Sequential_NLP/Fake News Challenge/')
os.chdir(project_path)
train_bodies = pd.read_csv('train_bodies.csv')
train_stances = pd.read_csv('train_stances.csv')

In [6]:
print(train_bodies.head(2))
print(train_bodies.shape)
print(train_bodies.info())

   Body ID                                        articleBody
0        0  A small meteorite crashed into a wooded area i...
1        4  Last week we hinted at what was to come as Ebo...
(1683, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1683 entries, 0 to 1682
Data columns (total 2 columns):
Body ID        1683 non-null int64
articleBody    1683 non-null object
dtypes: int64(1), object(1)
memory usage: 26.4+ KB
None


In [7]:
print(train_stances.head(2))
print(train_stances.shape)
print(train_stances.info())


                                            Headline  Body ID     Stance
0  Police find mass graves with at least '15 bodi...      712  unrelated
1  Hundreds of Palestinians flee floods in Gaza a...      158      agree
(49972, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49972 entries, 0 to 49971
Data columns (total 3 columns):
Headline    49972 non-null object
Body ID     49972 non-null int64
Stance      49972 non-null object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB
None


In [0]:
dataset= pd.merge(left=train_bodies, right=train_stances, on="Body ID", how="inner")

In [9]:
print(dataset.shape)
print(dataset.info())

(49972, 4)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49972 entries, 0 to 49971
Data columns (total 4 columns):
Body ID        49972 non-null int64
articleBody    49972 non-null object
Headline       49972 non-null object
Stance         49972 non-null object
dtypes: int64(1), object(3)
memory usage: 1.9+ MB
None



<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [10]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [13]:
from keras.preprocessing.text import Tokenizer
import keras
import keras.preprocessing

Using TensorFlow backend.


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [14]:
t = keras.preprocessing.text.Tokenizer(nb_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
        lower=True, split=" ")

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [15]:
t.fit_on_texts(dataset["articleBody"])

# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(len(t.word_counts))
print(t.word_index)
print(t.word_docs)

OrderedDict([('a', 471875), ('small', 4923), ('meteorite', 4597), ('crashed', 207), ('into', 24822), ('wooded', 554), ('area', 7121), ('in', 394397), ("nicaragua's", 403), ('capital', 6389), ('of', 432925), ('managua', 1483), ('overnight', 749), ('the', 1122054), ('government', 23143), ('said', 133986), ('sunday', 5416), ('residents', 3383), ('reported', 21615), ('hearing', 2460), ('mysterious', 725), ('boom', 1398), ('that', 232853), ('left', 11624), ('16', 2092), ('foot', 2312), ('deep', 1159), ('crater', 3166), ('near', 12473), ("city's", 726), ('airport', 5122), ('associated', 4582), ('press', 9548), ('reports', 22600), ('spokeswoman', 2460), ('rosario', 491), ('murillo', 1012), ('committee', 2417), ('formed', 650), ('by', 101215), ('to', 488107), ('study', 1432), ('event', 4817), ('determined', 1234), ('it', 124079), ('was', 177713), ('relatively', 703), ('appears', 6878), ('have', 108087), ('come', 8449), ('off', 14363), ('an', 90455), ('asteroid', 2587), ('passing', 1209), ('clo

In [16]:
t.fit_on_texts(dataset["Headline"])

# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(len(t.word_counts))
print(t.word_index)
print(t.word_docs)

OrderedDict([('a', 478544), ('small', 5273), ('meteorite', 5848), ('crashed', 207), ('into', 25604), ('wooded', 554), ('area', 7132), ('in', 408093), ("nicaragua's", 526), ('capital', 6867), ('of', 441773), ('managua', 1833), ('overnight', 749), ('the', 1129192), ('government', 23653), ('said', 134456), ('sunday', 5416), ('residents', 3428), ('reported', 21926), ('hearing', 2514), ('mysterious', 853), ('boom', 1459), ('that', 234452), ('left', 11643), ('16', 2172), ('foot', 2441), ('deep', 1218), ('crater', 3683), ('near', 12938), ("city's", 726), ('airport', 5598), ('associated', 4582), ('press', 9548), ('reports', 24361), ('spokeswoman', 2460), ('rosario', 491), ('murillo', 1012), ('committee', 2417), ('formed', 650), ('by', 104742), ('to', 499727), ('study', 1432), ('event', 4903), ('determined', 1234), ('it', 125904), ('was', 180100), ('relatively', 703), ('appears', 7092), ('have', 110415), ('come', 8595), ('off', 15561), ('an', 91223), ('asteroid', 2700), ('passing', 1254), ('clo

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
from nltk.tokenize import sent_tokenize, word_tokenize

texts = []  # list of text samples
articles = [] # list of articles sentences
texts = dataset["articleBody"].to_list()
for text in texts:
  articles.append(nltk.sent_tokenize(text))


## Check 2:

first element of texts and articles should be as given below. 

In [18]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [19]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence
import numpy as np

In [23]:
data = np.zeros((len(texts),MAX_SENTS, MAX_SENT_LENGTH))
for article_index, article in enumerate(articles):
  for sent_index, sent in enumerate(article):
    if sent_index <= MAX_SENTS-1:
        cur_words = text_to_word_sequence(sent)
    else:
        break;
    
    for word_index, word in enumerate(cur_words):
      if word_index <= MAX_SENT_LENGTH-1:
        data[article_index][sent_index][word_index] = t.word_index[word]
      else: 
        break;
      
print(data.shape) 

(49972, 20, 20)


### Check 3:

Accessing first element in data should give something like given below.

In [24]:
data[0, :, :]

array([[3.0000e+00, 4.8100e+02, 4.2700e+02, 7.2110e+03, 8.1000e+01,
        3.0000e+00, 3.7330e+03, 3.3100e+02, 5.0000e+00, 3.8910e+03,
        3.5000e+02, 4.0000e+00, 1.4310e+03, 2.9580e+03, 1.0000e+00,
        8.9000e+01, 1.2000e+01, 4.6400e+02, 0.0000e+00, 0.0000e+00],
       [7.5800e+02, 9.5000e+01, 1.0470e+03, 3.0000e+00, 2.6790e+03,
        1.7520e+03, 7.0000e+00, 1.8900e+02, 3.0000e+00, 1.2170e+03,
        1.0750e+03, 2.0300e+03, 7.0000e+02, 1.5900e+02, 1.0000e+00,
        3.0320e+03, 4.4800e+02, 1.0000e+00, 5.5500e+02, 2.3500e+02],
       [8.9000e+01, 1.0670e+03, 4.1150e+03, 2.3490e+03, 1.2000e+01,
        3.0000e+00, 1.0920e+03, 3.3060e+03, 1.9000e+01, 1.0000e+00,
        8.9000e+01, 2.0000e+00, 1.7930e+03, 1.0000e+00, 5.2100e+02,
        2.0090e+03, 1.5000e+01, 9.0000e+00, 3.0000e+00, 3.1110e+03],
       [1.8100e+02, 3.6400e+03, 9.7200e+02, 2.0000e+02, 2.5560e+03,
        4.4000e+01, 6.7750e+03, 1.7220e+03, 1.2520e+03, 5.0000e+00,
        1.3317e+04, 1.7936e+04, 1.0000e+00, 7


# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [25]:
texts_heading = []
articles_heading = []
texts_heading = dataset["Headline"].to_list()

for text in texts:
  articles_heading.append(nltk.sent_tokenize(text))
  
data_heading = np.zeros((len(texts),MAX_SENTS_HEADING, MAX_SENT_LENGTH))
for article_index, article in enumerate(articles_heading):
  for sent_index, sent in enumerate(article):
    if sent_index > MAX_SENTS_HEADING-1:
        break
    cur_words = text_to_word_sequence(sent)
    for word_index, word in enumerate(cur_words):
      if word_index > MAX_SENT_LENGTH-1:
        break
      data_heading[article_index][sent_index][word_index] = t.word_index[word]

print(data_heading.shape)

(49972, 1, 20)


### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
 labels = pd.get_dummies(dataset["Stance"])
 labels = np.array(labels)

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [27]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
#Get shuffled indices
indices = np.arange(data.shape[0])
np.random.shuffle(indices)

#Use shuffled indices to shuffle the data
data = data[indices]
data_heading = data_heading[indices]
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [29]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2)
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

x_heading_train, x_heading_val, y_train, y_val = train_test_split(data_heading, labels, test_size=0.2)
print(x_heading_train.shape, y_train.shape, x_heading_train.shape, y_val.shape)

(39977, 20, 20) (39977, 4) (9995, 20, 20) (9995, 4)
(39977, 1, 20) (39977, 4) (39977, 1, 20) (9995, 4)


### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [34]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977, 4)
(9995, 20, 20)
(9995, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [35]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('../glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


# create a weight matrix for words in training docs
embedding_matrix = np.zeros((len(t.word_index), 100))


for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


In [37]:
print(len(embeddings_index), embedding_matrix.shape, len(t.word_index) )

400000 (27873, 100) 27873


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [38]:
X_train = np.reshape(x_train, (x_train.shape[0], MAX_SENTS * MAX_SENT_LENGTH))
X_val = np.reshape(x_val, (x_val.shape[0], MAX_SENTS * MAX_SENT_LENGTH))
X_heading_train = np.reshape(x_heading_train, (x_heading_train.shape[0], MAX_SENTS_HEADING * MAX_SENT_LENGTH))
X_heading_val = np.reshape(x_heading_val, (x_heading_val.shape[0], MAX_SENTS_HEADING * MAX_SENT_LENGTH))
print(X_train.shape, y_train.shape, X_val.shape,y_val.shape)
print(X_heading_train.shape,y_train.shape, X_heading_val.shape,y_val.shape)


(39977, 400) (39977, 4) (9995, 400) (9995, 4)
(39977, 20) (39977, 4) (9995, 20) (9995, 4)


In [0]:
from keras.models import Model, Sequential
from keras.layers import Dense, BatchNormalization, Input, Concatenate, concatenate,Activation, Flatten, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPool1D, merge, dot
from keras.optimizers import SGD



### Model

In [0]:
head_input = Input(shape=(MAX_SENTS * MAX_SENT_LENGTH, ))
body_input = Input(shape=(MAX_SENTS_HEADING * MAX_SENT_LENGTH, ))

head_embed = Embedding(len(t.word_counts), 100, weights=[embedding_matrix])(head_input)
body_embed = Embedding(len(t.word_counts), 100, weights=[embedding_matrix])(body_input)

head_lstm = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(head_embed)
body_lstm = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(body_embed)

#head_lstm = Bidirectional(LSTM(100,dropout=0.2, recurrent_dropout=0.2, name='head_lstm'))(head_embed)
#body = Lambda(adder,output_shape = adder_output)(body_embed)
#dot_layer = dot([head_lstm,body_lstm],axes = 1, normalize=True)
#conc = concatenate([head_lstm,body_lstm])
#maxpool= GlobalMaxPool1D()(conc)
#dense1 = Dense(100,activation='relu')(maxpool)
#dropout = Dropout(0.3)(dense1)
#dense2 = Dense(4,activation='softmax')(dense1)
#model = Model(inputs=[head_input,body_input], outputs=[dense2])

head_maxpool = GlobalMaxPool1D()(head_lstm)
body_maxpool = GlobalMaxPool1D()(body_lstm)

head_dense = Dense(100, activation="relu")(head_maxpool)
head_dropout = Dropout(0.25)(head_dense)


body_dense = Dense(100, activation="relu")(body_maxpool)
body_dropout = Dropout(0.25)(body_dense)

comb = concatenate([head_dropout, body_dropout])
dense = Dense(4, activation="softmax")(comb)
model = Model(inputs=[head_input,body_input], outputs=dense)


### Compile and fit the model

In [47]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 400, 100)     2787300     input_9[0][0]                    
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 20, 100)      2787300     input_10[0][0]                   
____________________________________________________________________________________________

In [48]:
model.fit([X_train,X_heading_train], y_train, batch_size=128, epochs=6, validation_data=([X_val,X_heading_val], y_val))



Train on 39977 samples, validate on 9995 samples
Epoch 1/6
39977/39977 [==============================] - 489s 12ms/step - loss: 0.3311 - acc: 0.8676 - val_loss: 0.3067 - val_acc: 0.8770
Epoch 2/6
39977/39977 [==============================] - 490s 12ms/step - loss: 0.2861 - acc: 0.8854 - val_loss: 0.2686 - val_acc: 0.8908
Epoch 3/6
39977/39977 [==============================] - 491s 12ms/step - loss: 0.2599 - acc: 0.8971 - val_loss: 0.2497 - val_acc: 0.9025
Epoch 4/6
39977/39977 [==============================] - 488s 12ms/step - loss: 0.2453 - acc: 0.9031 - val_loss: 0.2495 - val_acc: 0.8974
Epoch 5/6
39977/39977 [==============================] - 487s 12ms/step - loss: 0.2351 - acc: 0.9060 - val_loss: 0.2373 - val_acc: 0.9063
Epoch 6/6
39977/39977 [==============================] - 486s 12ms/step - loss: 0.2297 - acc: 0.9074 - val_loss: 0.2365 - val_acc: 0.9069


In [65]:
dir()


['Activation',
 'BatchNormalization',
 'Bidirectional',
 'Concatenate',
 'Dense',
 'Dropout',
 'Embedding',
 'Flatten',
 'GlobalMaxPool1D',
 'In',
 'Input',
 'LSTM',
 'MAX_NB_WORDS',
 'MAX_SENTS',
 'MAX_SENTS_HEADING',
 'MAX_SENT_LENGTH',
 'Model',
 'Out',
 'SGD',
 'Sequential',
 'Tokenizer',
 'VALIDATION_SPLIT',
 'X_heading_train',
 'X_heading_val',
 'X_train',
 'X_val',
 'ZipFile',
 '_',
 '_10',
 '_12',
 '_18',
 '_19',
 '_24',
 '_48',
 '_62',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_exit_code',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i12',
 '_i13',
 '_i14',
 '_i15',
 '_i16',
 '_i17',
 '_i18',
 '_i19',
 '_i2',
 '_i20',
 '_i21',
 '_i22',
 '_i23',
 '_i24',
 '_i25',
 '_i26',
 '_i27',
 '_i28',
 '_i29',
 '_i3',
 '_i30',
 '_i31',
 '_i32',
 '_i33',
 '_i34',
 '_i35',
 '_i36',
 '_i37',
 '_i38',
 '_i39',
 '_i4',
 '_i40',
 '_i41',
 '_i42',
 '_i43',
 '_i44',
 '_i45',
 '_i46',
 '_i47',
 '_i48',
 '_i49',
 '_i5',
 '_i

In [64]:
#Utilities from original challenge: https://github.com/FakeNewsChallenge/fnc-1-baseline     
#There is a problem with the report_score hence the score line is showing incorrect value, but confusion matrix can be referred. 
os.chdir('/content/drive/My Drive/Colab Notebooks/Sequential_NLP/')
from fcnbaseline.utils import dataset, generate_test_splits, score
def evaluate_answer(model,model_inp, true):
    inv_category_dict = {0:'unrelated', 1: 'agree', 2: 'disagree', 3: 'discuss'}
    predicted = model.predict(model_inp)
    predicted = np.argmax(predicted,axis = 1)
    t = np.argmax(true,axis = 1)
    ground = list()
    pred = list()
    for i in predicted:
        pred.append(inv_category_dict[i])
    for i in t:
        ground.append(inv_category_dict[i])
    score.report_score(ground, pred)


evaluate_answer(model,[X_val[1:200],X_heading_val[1:200]],y_val[1:200])



-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     1     |     0     |     3     |     0     |
-------------------------------------------------------------
| disagree  |     0     |    14     |    23     |     4     |
-------------------------------------------------------------
|  discuss  |     0     |     1     |    142    |     1     |
-------------------------------------------------------------
| unrelated |     0     |     4     |     5     |     1     |
-------------------------------------------------------------
Score: 164.0 out of 191.5	(85.63968668407311%)


## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)